In [3]:
## Fuzzy Logic: Membership Functions
!pip install --user scikit-fuzzy


In [6]:
def fuzzy_efficiency_score(row):
    energy = row['ENERGY_CONSUMPTION_PER_SQFT']
    income = row['Pct_INCOME_MORE_THAN_150K']

    if energy < 35:
        energy_score = 1.0
    elif 35 <= energy <= 45:
        energy_score = 0.5
    else:
        energy_score = 0.0

    if income > 20:
        income_score = 1.0
    elif 10 < income <= 20:
        income_score = 0.5
    else:
        income_score = 0.0

    return round((energy_score + income_score) / 2 * 100, 2)


In [9]:
import pandas as pd

# Load the merged dataset
df = pd.read_csv("/workspaces/CPSC6185-Final-Project/data/processed/merged_cleaned.csv")

# Apply fuzzy efficiency score function
df['EFFICIENCY_SCORE'] = df.apply(fuzzy_efficiency_score, axis=1)

# Save the result (optional but recommended)
df.to_csv("/workspaces/CPSC6185-Final-Project/data/processed/merged_cleaned.csv", index=False)

# Preview
df[['STATE', 'ENERGY_CONSUMPTION_PER_SQFT', 'Pct_INCOME_MORE_THAN_150K', 'EFFICIENCY_SCORE']].head()


,STATE,ENERGY_CONSUMPTION_PER_SQFT,Pct_INCOME_MORE_THAN_150K,EFFICIENCY_SCORE
0,Colorado,43.74,15.86,50.0
1,Utah,43.08,11.72,50.0
2,Idaho,45.09,7.67,0.0
3,Montana,46.71,6.11,0.0
4,North Dakota,46.75,12.24,25.0
